## Import thư viện

In [1]:
import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io


In [2]:
import torch
import torchvision
import cv2

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\ADMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-9-17 Python-3.11.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

# Set seed for random in numpy
np.random.seed(0) 

## Assignment fuction

In [4]:
def linear_assignment(cost_matrix):
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x,y)))

### Test

In [6]:
# Test
cost_matrix = [[1,2,3], [2,1,4], [3, 1, 1]]
print(linear_assignment(cost_matrix))

[[0 0]
 [1 1]
 [2 2]]


## Tính IOU

In [5]:
def iou_batch(bb_test, bb_gt):
    '''
        Compute iou between 2 bboxes in form [x1,y1,x2,y2]
    '''
    bb_gt = np.expand_dims(bb_gt, 0) #add 1 dim before col 0
    bb_test = np.expand_dims(bb_test, 1)
    
    # Tìm tọa đô giao nhau
    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    
    # Chiều cao, rộng giao nhau
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    
    # Diện tích giao nhau
    wh = w * h
    o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
    return(o)  

### Test

In [8]:
bb_test = np.array([
    [0, 0, 2, 2],  
    [1, 1, 3, 3]   
])

bb_gt = np.array([
    [0, 0, 1, 1],  
    [1, 1, 2, 2]  
])

print(iou_batch(bb_test, bb_gt))

[[       0.25        0.25]
 [          0        0.25]]


## Chuyển định dạng Bounding box

In [6]:
def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))

### Test

In [10]:
bbox_test = np.array([2, 3, 5, 7])
print(convert_bbox_to_z(bbox_test))

[[        3.5]
 [          5]
 [         12]
 [       0.75]]


In [7]:
def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

## Class Kalman bounding box tracker

In [8]:
class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
# ID duy nhất cho đối tượng
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    # xk = [x,y,s,r,vx,vy,vs]
    # zk = [x1, y1, x2, y2]
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    # Ma trận biến đổi trạng thái
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    # Ma trận đo lường
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    
    # Ma trận phương sai
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    
    # Ma trận nhiễu quá trình
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    # Khởi tạo giá trị x
    self.kf.x[:4] = convert_bbox_to_z(bbox)
    
    # Số frame đã qua kể từ lần cuối update
    self.time_since_update = 0
    
    # Gán ID độc nhất
    self.id = KalmanBoxTracker.count
    
    # Tăng ID lên 1 cho những đối tượng tiếp
    KalmanBoxTracker.count += 1
    
    # Lưu lại lịch sử các bounding box
    self.history = []
    
    # Số lần update thành công
    self.hits = 0
    
    # Số lần update thành công liên tiếp
    self.hit_streak = 0
    
    # Số frames đã qua kể từ khi tạo
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    # Ngăn không cho giá trị vận tốc về âm
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    
    self.kf.predict()
    self.age += 1
    
    # Nếu không có update nào trong 1 khoảng thời gian
    if(self.time_since_update>0):
      self.hit_streak = 0
    
    self.time_since_update += 1
    
    # Thêm bounding box dự đoán được vào lịch sử
    self.history.append(convert_x_to_bbox(self.kf.x))
    
    # Trả về bounding box vừa dự đoán
    return self.history[-1]

  # Trả về trạng thái x
  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)

## Liên kết detection với track

In [9]:
def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  # Nếu không có track nào: Danh sách đã liên kết rỗng, danh sách detection chưa liên kết giữ nguyên, danh sách các track rỗng
  # Danh sách liên kết giữa track và detection bao gồm 2 phần tử tương ứng chỉ số của detection và track (i, j)
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  # Tính giá trị IOU giữa các detection và track
  iou_matrix = iou_batch(detections, trackers)

  # Liên kết
  # Kiểm tra ma trận có hợp lệ không
  if min(iou_matrix.shape) > 0:
    # Tạo ma trận nhị phân, nếu > ngưỡng = 1, ngược lại = 0
    a = (iou_matrix > iou_threshold).astype(np.int32)
    # mỗi i chỉ có duy nhất 1 j
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  # Danh sách những detection chưa được liên kết
  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
    
  # Danh sách các track chưa liên kết
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

### Test

In [15]:
detection = np.array([[1,1,3,3],
                     [5, 5, 7, 7]])
track = np.array([[1.1,1.2,3.1,3.2],
                 [8, 8, 9, 9]])
matches, unmatchDetection, unmatchTrack = associate_detections_to_trackers(detection, track, iou_threshold=0.3)
print("Match track :")
print(matches)
print("Unmatch Detection")
print(unmatchDetection)
print("Unmatch Track")
print(unmatchTrack)

Match track :
[[0 0]]
Unmatch Detection
[1]
Unmatch Track
[1]


## Class SORT

In [10]:
class Sort(object):
  def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
    """
    Sets key parameters for SORT
    """
    # Số frame tối đa kể từ lần cuối đối tượng được track trước khi xóa
    self.max_age = max_age
    # Số frame tối thiểu để một track thành "confirmed"
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0

  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    
    # get predicted locations from existing trackers.
    # Lưu danh sách dự đoán các track
    trks = np.zeros((len(self.trackers), 5))
    # Xóa track lỗi
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      # Dự đoán tracks thứ t
      pos = self.trackers[t].predict()[0]
      # trk có dạng [x1,y1,x2,y2, score]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      # Nếu có 1 giá trị của track = NaN --> xóa
      if np.any(np.isnan(pos)):
        to_del.append(t)
      
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    
    # Liên kết
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0]])

    # print(unmatched_dets)
      
    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        # print(dets[i])
        trk = KalmanBoxTracker(dets[i])
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

## DEMO 

In [11]:
def Object_Detection(frame):
    results = model(frame)
    
    # Get bounding boxes, class names, and confidence scores
    detections = results.xyxy[0].cpu().numpy()
    
    list_detection = []
    
    for detection in detections :
        xmin, ymin, xmax, ymax, confidence, obj_id = detection[:6]
        # Chỉ detect người
        if obj_id != 0: continue
        else :
            list_detection.append([xmin, ymin, xmax, ymax, confidence])
    
    return list_detection

In [15]:
def main(vid_path, output_path):
    
    # Open video file
    cap = cv2.VideoCapture(vid_path)
    
    # Check if the video was opened correctly
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') 
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Create a VideoWriter object
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    tracker = Sort()
    while True:
        ret, frame = cap.read()
        
        if not(ret) : 
            break
        
        # Detect Object
        detections = Object_Detection(frame)
        
        # print(detections)
        
        # Update the SORT tracker with the detected bounding boxes
        tracked_objects = tracker.update(detections)
        
        # Visualize the tracked objects
        for obj in tracked_objects:
            x1, y1, x2, y2, obj_id = obj
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # Draw the object ID
            cv2.putText(frame, f"ID: {int(obj_id)}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
        
         # Show the frame with tracking
        cv2.imshow('SORT Tracking', frame)
        
        out.write(frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [16]:
if __name__ == "__main__":
    vid_path = r'E:\Project\SamSungR&D\Computer Vision\Multiple Object Tracking\vid\Venice-2.mp4'
    output_path = r'E:\Project\SamSungR&D\Computer Vision\Multiple Object Tracking\vid\Output-Venice-2.mp4'
    main(vid_path, output_path)